# Event Management Document Generation
In this notebook, we'll be demonstrating how to create a corpus of documents quickly with a little bit of Python and help from a large language model (LLM). For my own purposes, I am simulating a corpus of documents around a **fake event management company**.

You'll see that we will make use of the **OpenAI API**, so you'll need to set yourself up with an OpenAI API key if you'd like to use this precise code here. But this framework is really compatible with any model API. In the "Notebook Setup" section down below, you'll see a simple function called `generate_response()`. All you'd need to do to make this whole notebook compatible with your generation mechanism of choice is update this function!

## Notebook Setup
In this section, we will import the necessary Python libraries and set up our LLM API. By default, I am going with OpenAI's API, and I am loading my personal API key from a sensitive file. As a reminder, if you would like to use a different LLM generation source, you can update the `generate_response()` function with your own code.

In [1]:
import os, json, yaml
from openai import OpenAI

In [2]:
# Loading my private OpenAI API key from file (NOT pushed to GitHub, of course!)
with open('../../keys/api_keys.yaml', 'r') as f:
    API_KEYS = yaml.safe_load(f)

# Setting the OpenAI client
openai_client = OpenAI(api_key = API_KEYS['OPENAI_API_KEY'])

In [3]:
def generate_response(prompt):
    '''
    Generates a response from an LLM API. (Default: OpenAI)

    Inputs:
        - prompt (str): The input prompt

    Returns:
        - (str): The generated response
    '''

    # Getting the generated response from the OpenAI API
    response = openai_client.chat.completions.create(
        model = "gpt-5",
        messages = [
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

In [4]:
# Creating the prompt template for event management topic generation
EVENT_MGMT_TOPIC_GENERATION_PROMPT = '''
You are acting as a domain expert in event planning and operations.  
Generate 7 distinct **topics** that represent major areas of knowledge required to plan and execute a large-scale event, such as a conference, product launch, or corporate summit.  

For each topic:
- Generate 8 unique knowledge items.
- Each knowledge item should be represented as a JSON object with two keys:
  - "doc_title": A short, descriptive title for the knowledge item.
  - "doc_description": A clear description of the content, purpose, and potential format of this knowledge item.

Output the results as a valid JSON array, with each topic structured as:
{
  "topic": "<Topic Name>",
  "documents": [
    {
      "doc_title": "<Title>",
      "doc_description": "<Description>"
    },
    ...
  ]
}

Ensure that:
- The JSON is syntactically valid.
- All topics are unique and cover different areas of event planning.
- Each "doc_description" is specific and useful for understanding the document's role in the planning process.

Only include the JSON. Do not include any additional text, such as "Here are your results"
'''

In [5]:
# Setting the filepath for caching the generated event management topics
event_mgmt_topics_filepath = 'cache/event_mgmt_topics.json'

# Checking to see if the topic file has already been created
if not os.path.exists(event_mgmt_topics_filepath):

    # Generating the response using the OpenAI API
    response = generate_response(EVENT_MGMT_TOPIC_GENERATION_PROMPT)

    # Loading the topic generation from the response
    event_mgmt_topics_json = json.loads(response)

    # Writing the JSON out to the cache filepath
    with open(event_mgmt_topics_filepath, 'w') as outfile:
        json.dump(event_mgmt_topics_json, outfile, indent = 2)
else:
    print('Topics already generated. Loading from file.')

    # Loading the JSON from the cache filepath
    with open(event_mgmt_topics_filepath, 'r') as infile:
        event_mgmt_topics_json = json.load(infile)

Topics already generated. Loading from file.


In [6]:
# Creating the event management document generation prompt
EVENT_MGMT_DOC_GENERATION_PROMPT = '''
You are acting as an expert event planner and technical writer.  
Your task is to create a **robust, standalone knowledge item** based on the following input:

- Topic: {topic}  
- Document Title: {doc_title}  
- Document Description: {doc_description}

Requirements for the document:
- Write in a clear, professional, and instructional tone.
- Include enough depth so that a team member with no prior context could immediately understand and act on this document.
- Where applicable, include structured sections such as:
  - **Overview**
  - **Key Details or Steps**
  - **Responsibilities**
  - **Dependencies**
  - **References or Templates**
- Output should be formatted in Markdown.
- Make the content specific and realistic — include example data, timelines, or lists where appropriate.
'''

In [7]:
# Setting the base data directory
base_data_dir = '../data/event_mgmt_docs'

# Creating the directory if it doesn't exist
os.makedirs(base_data_dir, exist_ok = True)

# Iterating over all the topic groupings
for topic_group in event_mgmt_topics_json:
    topic = topic_group['topic']
    documents = topic_group['documents']

    # Creating a directory for the topic
    topic_dir = os.path.join(base_data_dir, topic.replace(' ', '_'))
    os.makedirs(topic_dir, exist_ok = True)

    # Iterating over each document in the topic
    for doc in documents:
        doc_title = doc['doc_title']
        doc_description = doc['doc_description']

        # Set the output filepath for this document
        filename = f"{doc_title.replace(' ', '_')}.md"
        filepath = os.path.join(topic_dir, filename)

        # Only generate and write if the file does not already exist
        if not os.path.exists(filepath):

            # Generating the document content using the OpenAI API
            content = generate_response(EVENT_MGMT_DOC_GENERATION_PROMPT.format(topic = topic, doc_title = doc_title, doc_description = doc_description))

            # Writing the content to a Markdown file
            with open(filepath, 'w') as f:
                f.write(content)
        else:
            print(f"File already exists: {filepath}")

File already exists: ../data/event_mgmt_docs/Event_Strategy_&_Stakeholder_Alignment/Event_Purpose_&_Success_Metrics_Framework.md
File already exists: ../data/event_mgmt_docs/Event_Strategy_&_Stakeholder_Alignment/Stakeholder_Map_&_RACI_Matrix.md
File already exists: ../data/event_mgmt_docs/Event_Strategy_&_Stakeholder_Alignment/Audience_Personas_&_Value_Proposition.md
File already exists: ../data/event_mgmt_docs/Event_Strategy_&_Stakeholder_Alignment/Event_Narrative_&_Messaging_Architecture.md
File already exists: ../data/event_mgmt_docs/Event_Strategy_&_Stakeholder_Alignment/Business_Case_&_ROI_Model.md
File already exists: ../data/event_mgmt_docs/Event_Strategy_&_Stakeholder_Alignment/Timeline_&_Governance_Cadence.md
File already exists: ../data/event_mgmt_docs/Event_Strategy_&_Stakeholder_Alignment/Risk_Appetite_&_Decision_Escalation_Framework.md
File already exists: ../data/event_mgmt_docs/Event_Strategy_&_Stakeholder_Alignment/Post-Event_Impact_Measurement_Plan.md
File already exi